In [12]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [3]:
pyspark.__version__

'3.3.2'

In [4]:
pyspark.__file__

'/home/rudolph/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-03 10:32:54--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.160, 3.164.82.112, 3.164.82.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  19.0MB/s    in 4.1s    

2025-03-03 10:32:59 (15.1 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework_5') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/03 10:33:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2025-03-02 08:14:42--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.40, 3.164.82.197, 3.164.82.160, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-02 08:14:43 (229 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [11]:
!head taxi_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [18]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [8]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')


In [9]:
df_yellow.repartition(4).write.parquet('yellow_tripdata_2024_10_partitioned')


In [14]:
df_yellow.columns
# df_yellow.select('tpep_pickup_datetime').show()

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge',
 'Airport_fee']

In [13]:


df_yellow \
    .filter(F.to_date(df_yellow.tpep_pickup_datetime) == '2024-10-15') \
    .count()


128893

In [16]:
df_yellow \
    .select(
        'trip_distance',
        'tpep_pickup_datetime', 
        'tpep_dropoff_datetime',
        ((F.unix_timestamp('tpep_dropoff_datetime') - F.unix_timestamp('tpep_pickup_datetime')) / 3600).alias('duration_hours')
    ) \
    .orderBy(F.desc('trip_distance')) \
    .limit(1) \
    .show()


+-------------+--------------------+---------------------+------------------+
|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|    duration_hours|
+-------------+--------------------+---------------------+------------------+
|    366343.04| 2024-10-16 17:24:00|  2024-10-16 17:59:00|0.5833333333333334|
+-------------+--------------------+---------------------+------------------+



In [17]:
df_yellow \
    .select(
        'trip_distance',
        'tpep_pickup_datetime', 
        'tpep_dropoff_datetime',
    ) \
    .orderBy(F.desc('trip_distance')) \
    .show()


+-------------+--------------------+---------------------+
|trip_distance|tpep_pickup_datetime|tpep_dropoff_datetime|
+-------------+--------------------+---------------------+
|    366343.04| 2024-10-16 17:24:00|  2024-10-16 17:59:00|
|    331688.82| 2024-10-03 06:52:00|  2024-10-03 07:01:00|
|    319111.84| 2024-10-15 23:54:00|  2024-10-16 00:02:00|
|    318500.89| 2024-10-10 20:42:00|  2024-10-10 20:51:00|
|    222376.21| 2024-10-04 19:10:00|  2024-10-04 19:24:00|
|    160244.91| 2024-10-18 05:58:48|  2024-10-18 07:16:47|
|    151827.64| 2024-10-15 10:17:00|  2024-10-15 10:26:00|
|    148154.85| 2024-10-18 05:41:00|  2024-10-18 05:44:00|
|    126036.33| 2024-10-10 23:01:00|  2024-10-10 23:13:00|
|    112006.79| 2024-10-30 07:12:00|  2024-10-30 07:34:00|
|    109761.71| 2024-10-15 07:38:00|  2024-10-15 07:47:00|
|    108773.45| 2024-10-11 05:54:00|  2024-10-11 06:02:00|
|     103191.8| 2024-10-14 08:09:00|  2024-10-14 08:31:00|
|    102727.94| 2024-10-08 08:06:00|  2024-10-08 08:30:0

In [19]:
# Read the zone lookup data
# df_zones = spark.read.parquet('zones/')

# Join yellow taxi data with zones on pickup location
pickup_counts = df_yellow \
    .groupBy('PULocationID') \
    .count() \
    .join(df_zones, df_yellow.PULocationID == df_zones.LocationID) \
    .select('Zone', 'count') \
    .orderBy('count') \
    .limit(1)

pickup_counts.show()


+--------------------+-----+
|                Zone|count|
+--------------------+-----+
|Governor's Island...|    1|
+--------------------+-----+

